![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
# Chapter 4: Allocating AoAs to blades

## Dependencies

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

## Imports

In [ ]:
from bladesight import Datasets
from bladesight.btt.aoa import transform_ToAs_to_AoAs
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numba import njit
import pandas as pd
import numpy as np
from typing import Tuple, List

## Generated AoAs

In [ ]:
delta = -20
blade_means = np.deg2rad([72, 144+delta, 216+delta, 288+delta, 360+delta])
aoa_values = []
BLADE_COUNT = 5
# Set random seed for reproducibility
np.random.seed(0)
for n in range(50):
    r = 0.5 + 0.5/50 * n
    for b in range(BLADE_COUNT):
        aoa_current = blade_means[b] + np.random.uniform(-np.pi*0.07, np.pi*0.07)
        # Reject values with probability < 0.05
        if np.random.rand() > 0.05:
            aoa_values.append({
                "n" : n,
                "aoa" : aoa_current,
                "plot_r" : r,
            })
df_aoas = pd.DataFrame(aoa_values)

## Unallocated AoAs

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=df_aoas["plot_r"],
    theta=df_aoas["aoa"]*180/np.pi,
    mode='markers',
    name='Unallocated AoAs'
))
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0, 1]
        )),
    showlegend=True
)
fig.update_polars(
    angularaxis=dict(direction="clockwise")
)

## Sequential approach

In [ ]:
df_aoa_sequential = df_aoas.copy(deep=True)
df_aoa_sequential['blade'] = None
df_aoa_sequential.loc[::5, 'blade'] = 1
df_aoa_sequential.loc[1::5, 'blade'] = 2
df_aoa_sequential.loc[2::5, 'blade'] = 3
df_aoa_sequential.loc[3::5, 'blade'] = 4
df_aoa_sequential.loc[4::5, 'blade'] = 5

In [ ]:
fig = go.Figure()
markers = ["circle", "square", "diamond", "cross", "x"]
for b in range(BLADE_COUNT):
    df_blade = df_aoa_sequential[df_aoa_sequential["blade"] == (b+1)]
    fig.add_trace(go.Scatterpolar(
        r=df_blade["plot_r"],
        theta=df_blade["aoa"]*180/np.pi,
        mode='markers',
        name=f'Blade {b+1}',
        marker_symbol=markers[b]
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0, 1]
        )),
    showlegend=True
)
fig.update_polars(
    angularaxis=dict(direction="clockwise")
)


## Binning approach

In [ ]:
df_aoa_binned = df_aoas.copy(deep=True)
df_aoa_binned['blade'] = None
bin_edges = [0, 72, 144, 216, 288]
bin_edges_rad = np.deg2rad(bin_edges)
for b, bin_left_edge in enumerate(bin_edges_rad):
    bin_right_edge = bin_left_edge + 2*np.pi/5
    ix_in_bin = (
        (df_aoa_binned['aoa'] >= bin_left_edge) 
        & (df_aoa_binned['aoa'] < bin_right_edge)
    )
    df_aoa_binned.loc[ix_in_bin, 'blade'] = b + 1

In [ ]:
fig = go.Figure()
markers = ["circle", "square", "diamond", "cross", "x"]
for b in range(BLADE_COUNT):
    df_blade = df_aoa_binned[df_aoa_binned["blade"] == (b+1)]
    fig.add_trace(go.Scatterpolar(
        r=df_blade["plot_r"],
        theta=df_blade["aoa"]*180/np.pi,
        mode='markers',
        name=f'Blade {b+1}',
        marker_symbol=markers[b]
    ))
# Add the bin edges as black dashed lines
for b in range(BLADE_COUNT):
    if b > 0:
        fig.add_trace(go.Scatterpolar(
            r=[0, 1],
            theta=[bin_edges[b]]*2,
            mode='lines',
            line=dict(width=2, dash='dash', color='black'),
            showlegend=False
        ))
    else:
        fig.add_trace(go.Scatterpolar(
            r=[0, 1],
            theta=[bin_edges[b]]*2,
            mode='lines',
            line=dict(width=2, dash='dash', color='black'),
            name='Bin edges'
        ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0, 1]
        )),
    showlegend=True
)
fig.update_polars(
    angularaxis=dict(direction="clockwise")
)


## Rotated binning approach

In [ ]:
df_aoa_rotated_binned = df_aoas.copy(deep=True)
df_aoa_rotated_binned['blade'] = None
bin_edges_new = [-36, 36, 108, 180, 252]
bin_edges_new_rad = np.deg2rad(bin_edges_new)
for b, bin_left_edge in enumerate(bin_edges_new_rad):
    bin_right_edge = bin_left_edge + 2*np.pi/5
    ix_in_bin = (
        (
                (df_aoa_rotated_binned['aoa'] >= bin_left_edge) 
                & (df_aoa_rotated_binned['aoa'] < bin_right_edge)
        )
        | (
                ((df_aoa_rotated_binned['aoa'] - 2*np.pi) >= bin_left_edge) 
                & ((df_aoa_rotated_binned['aoa'] - 2*np.pi) < bin_right_edge)
        )
    )
    df_aoa_rotated_binned.loc[ix_in_bin, 'blade'] = b + 1

In [ ]:
fig = go.Figure()
markers = ["circle", "square", "diamond", "cross", "x"]
for b in range(BLADE_COUNT):
    df_blade = df_aoa_rotated_binned[df_aoa_rotated_binned["blade"] == (b + 1)]
    fig.add_trace(go.Scatterpolar(
        r=df_blade["plot_r"],
        theta=df_blade["aoa"]*180/np.pi,
        mode='markers',
        name=f'Blade {b+1}',
        marker_symbol=markers[b]
    ))
# Add the bin edges as black dashed lines
for b in range(BLADE_COUNT):
    if b > 0:
        #fig.add_trace(go.Scatterpolar(
        #    r=[0, 1],
        #    theta=[bin_edges[b]]*2,
        #    mode='lines',
        #    line=dict(width=2, dash='dash', color='black'),
        #    showlegend=False
        #))
        fig.add_trace(go.Scatterpolar(
            r=[0, 1],
            theta=[bin_edges_new[b]]*2,
            mode='lines',
            line=dict(
                width=2, 
                dash= 'dot',
                color='green'
            ),
            showlegend=False
        ))
    else:
        #fig.add_trace(go.Scatterpolar(
        #    r=[0, 1],
        #    theta=[bin_edges[b]]*2,
        #    mode='lines',
        #    line=dict(width=2, dash='dash', color='black'),
        #    name='Old bin edges'
        #))
        fig.add_trace(go.Scatterpolar(
            r=[0, 1],
            theta=[bin_edges_new[b]]*2,
            mode='lines',
            line=dict(
                width=2, 
                dash= 'dot',
                color='green'
            ),
            name='Rotated bin edges'
        ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0, 1]
        )),
    showlegend=True
)
fig.update_polars(
    angularaxis=dict(direction="clockwise")
)


## Load the data

In [ ]:
dataset = Datasets['data/intro_to_btt/intro_to_btt_ch03']
df_opr_zero_crossings = dataset[f"table/du_toit_2017_test_1_opr_zero_crossings"]
df_prox_toas = dataset[f"table/du_toit_2017_test_1_prox_1_toas"]
df_prox_1 = transform_ToAs_to_AoAs(df_opr_zero_crossings, df_prox_toas)

In [ ]:
def calculate_Q(
    arr_aoas : np.ndarray,
    d_theta : float,
    N : int
) -> Tuple[float, np.ndarray]:
    bin_edges = np.linspace(0 + d_theta, 2*np.pi + d_theta, N + 1)
    Q = 0
    for b in range(N):
        left_edge = bin_edges[b]
        right_edge = bin_edges[b + 1]
        bin_mask = (arr_aoas > left_edge) & (arr_aoas <= right_edge)

        bin_centre = (left_edge + right_edge)/2
        Q += np.sum(
            (
                arr_aoas[bin_mask] 
                - bin_centre
            )**2 
        )
    if np.sum(arr_aoas < bin_edges[0]) > 0:
        return np.nan, bin_edges
    if np.sum(arr_aoas > bin_edges[-1]) > 0:
        return np.nan, bin_edges
    return Q, bin_edges


## Implementation example

In [ ]:
B = 5
d_thetas = np.linspace(-np.pi/B, np.pi/B, 200) 
arr_aoas = df_prox_1["AoA"].to_numpy()
Qs = [] 
optimal_Q, optimal_bin_edges, optimal_d_theta = np.inf, None, None
for d_theta in d_thetas:
    Q, bin_edges = calculate_Q(arr_aoas, d_theta, B)
    if Q < optimal_Q:
        optimal_Q = Q*1
        optimal_bin_edges = bin_edges
        optimal_d_theta = d_theta*1
    Qs.append(Q)


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=d_thetas * 180/np.pi,
        y=Qs,
        name="Q values"
    )
)
fig.add_trace(
    go.Scatter(
        x=[optimal_d_theta * 180/np.pi],
        y=[optimal_Q],
        name="Optimal d_theta value= {:.2f}°".format(optimal_d_theta*180/np.pi),
        mode="markers",
        marker={
            "size": 10
        }
    )
)

fig.update_layout(
    title="Q values for different d_theta values",
    xaxis_title="d_theta [deg]",
    yaxis_title="Q"
)
fig.show()


## Blade DataFrames


In [ ]:
blade_dfs = []
for b in range(B):
    ix_bin = (
        (df_prox_1["AoA"] > optimal_bin_edges[b])
        & (df_prox_1["AoA"] <= optimal_bin_edges[b + 1])
    )
    blade_dfs.append(
        df_prox_1.loc[ix_bin]
    )


In [ ]:
for b in range(B):
    print(f"Blade {b} mean: {blade_dfs[b]['AoA'].mean()}, std: {blade_dfs[b]['AoA'].std()}")

## Wrapping blades

In [ ]:
df_prox_1_shifted = df_prox_1.copy(deep=True)
df_prox_1_shifted['AoA'] = df_prox_1_shifted['AoA'] - 0.280844143512115
df_prox_1_shifted['AoA'] = df_prox_1_shifted['AoA'] % (2*np.pi)

B = 5
d_thetas = np.linspace(-np.pi/B, np.pi/B, 200)
arr_aoas = df_prox_1_shifted["AoA"].to_numpy()
Qs = []
optimal_Q, optimal_bin_edges, optimal_d_theta = np.inf, None, None
for d_theta in d_thetas:
    Q, bin_edges = calculate_Q(arr_aoas, d_theta, B)
    if Q < optimal_Q:
        optimal_Q = Q*1
        optimal_bin_edges = bin_edges
        optimal_d_theta = d_theta*1
    Qs.append(Q)


In [ ]:
print(optimal_Q)
print(optimal_bin_edges)


## Coding exercises

In [ ]:
# Your turn 👇
def calculate_Q(
    arr_aoas : np.ndarray,
    d_theta : float,
    N : int
) -> Tuple[float, np.ndarray]:
    # Please complete me
    ...

In [ ]:
# Your turn 👇
def transform_prox_AoAs_to_blade_AoAs(
    df_prox : pd.DataFrame,
    B : int,
) -> List[pd.DataFrame]:
    # Please complete me
    ...